In [11]:
import os
import pandas as pd
import re

# --- 설정 ---
# 텍스트 파일이 들어있는 폴더 이름
INPUT_DIR = 'pneumo_study'
# 결과를 저장할 CSV 파일 이름
OUTPUT_CSV = 'pneumo_findings_with_filename.csv'

# --- 메인 로직 ---
def extract_findings_data():
    """지정된 폴더에서 txt파일을 읽어 studyID, FINDINGS, filename을 추출하고 CSV로 저장합니다."""
    
    # 1. 폴더 존재 여부 확인
    if not os.path.isdir(INPUT_DIR):
        print(f"❌ 오류: '{INPUT_DIR}' 폴더를 찾을 수 없습니다.")
        return

    print(f"📁 '{INPUT_DIR}' 폴더에서 데이터 추출을 시작합니다...")
    
    # 추출한 데이터를 저장할 리스트
    extracted_data = []

    # 2. 폴더 내의 모든 파일 순회
    for filename in os.listdir(INPUT_DIR):
        if filename.endswith('.txt'):
            file_path = os.path.join(INPUT_DIR, filename)
            
            try:
                # --- A. 파일명에서 studyID 추출 ---
                study_id = filename.removeprefix('s').removesuffix('.txt')

                # --- B. 파일 내용에서 FINDINGS 부분 추출 ---
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                match = re.search(r'FINDINGS:(.*?)IMPRESSION:', content, re.DOTALL | re.IGNORECASE)
                
                findings_text = ""
                if match:
                    findings_text = match.group(1).strip()
                else:
                    match_until_end = re.search(r'FINDINGS:(.*)', content, re.DOTALL | re.IGNORECASE)
                    if match_until_end:
                        findings_text = match_until_end.group(1).strip()
                    else:
                        findings_text = "Not Found"

                # --- C. 추출한 데이터를 딕셔너리로 묶어 리스트에 추가 ---
                # 'filename' 키와 값을 추가했습니다.
                extracted_data.append({
                    'studyID': study_id,
                    'FINDINGS': findings_text,
                    'filename': filename  # <--- 파일명 컬럼 추가
                })
                # print(f"📄 파일: {filename} -> studyID: {study_id} (추출 완료)")

            except Exception as e:
                print(f"❌ 파일 처리 중 오류 발생: {filename}, 오류: {e}")

    # 3. 리스트를 Pandas DataFrame으로 변환
    if not extracted_data:
        print("처리할 텍스트 파일이 없습니다.")
        return
        
    # 컬럼 순서 지정
    # df = pd.DataFrame(extracted_data, columns=['studyID', 'filename', 'FINDINGS'])
    df = pd.DataFrame(extracted_data, columns=['studyID', 'FINDINGS'])

    # 4. DataFrame을 CSV 파일로 저장
    try:
        df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
        print(f"\n✨ 모든 작업 완료! 결과가 '{OUTPUT_CSV}' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"❌ CSV 파일 저장 중 오류 발생: {e}")


# # --- 스크립트 실행 ---
# if __name__ == "__main__":
#     extract_findings_data()

In [12]:
extract_findings_data()

📁 'pneumo_study' 폴더에서 데이터 추출을 시작합니다...

✨ 모든 작업 완료! 결과가 'pneumo_findings_with_filename.csv' 파일로 저장되었습니다.


In [13]:
df = pd.read_csv(OUTPUT_CSV)
not_found_df = df[df['FINDINGS'] == 'Not Found']

In [16]:
not_found_df.count()

studyID     665
FINDINGS    665
dtype: int64

In [17]:
df.count()

studyID     2844
FINDINGS    2843
dtype: int64

In [18]:
finding_df = df[df['FINDINGS'] != 'Not Found']
finding_df.to_csv('pneumo_findings_only.csv', index=False, encoding='utf-8-sig')
not_found_df.to_csv('pneumo_not_found.csv', index=False, encoding='utf-8-sig')